### Requirements and Google Colab setup
The requirements that you will need for this assigment are:

- If you work on Google Colab:
    - tensorboardX
    - a small utility called `ngrok` that let you see the Tensorboard panel in a separate webpage
- If you use your local GPU with your local environment:
    - (PyTorch)
    - tensorboardX
    - tensorboard (included if you already have tensorflow)

In [ ]:
#### RUN THIS CODE IF YOU USE GOOGLE COLAB ####

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the material for this assignment,
# e.g. 'cvf20/assignments/assignment3/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Make sure that the python modules in the assignment folder are found by the notebook:
import sys
import os
path_drive = os.path.join("/content/drive/My Drive", FOLDERNAME)
sys.path.append(path_drive)

# Copy the yeast-cells data in the content folder of the notebook:
dataset_path = os.path.join(path_drive, "yeast_cells_dataset") 
dataset_path = dataset_path.replace(" ", "\ ")
!cp -r $dataset_path ./

# Move to the main content folder:
%cd /content

# Install tensorboardX:
!pip install tensorboardX

In [ ]:
#### RUN THIS CODE IF YOU USE GOOGLE COLAB OR IF YOU WANT TO USE THE NGROK UTILITY ####

! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

# 1. Train a CNN for Semantic Segmentation (Part 2)

In [ ]:
# The usual imports:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.ndimage
plt.rcParams['figure.figsize'] = [15, 15]

import torch
from torch import nn
import h5py
import os

### a) Loss functions for semantic segmentation
In the code block below, implement the Dice loss defined in the description of the assignment. We will implement it as a subclass of `torch.nn.Module`, which is the base PyTorch class for all neural network modules. The method `forward()` is the one that performs the forward step, i.e. computes the output of the layer from the given inputs. 

We don't need to implement Binary Cross Entropy, since it is already implemented in PyTorch.

In [ ]:
class SorensenDiceLoss(nn.Module):
    """
    Computes a loss scalar, which when minimized maximizes the Sorensen-Dice similarity
    between the input and the target.
    """
    def __init__(self, eps=1e-6):
        super(SorensenDiceLoss, self).__init__()
        self.eps = eps

    def forward(self, input, target):
        """
        input:      torch.FloatTensor with float values between 0 and 1
        target:     torch.FloatTensor with binary values 0 and 1

        Shape of the inputs: (batch_size, 1, x_size_image, y_size_image)
        
        When you divide by the denominator in the Dice loss formula, you can use the `eps` parameter and the
        `clamp` method to avoid a division by zero:
        
         loss = 1 - 2 * (numerator / denominator.clamp(min=self.eps))
        
        """
        assert input.shape == target.shape
        loss = torch.zeros((1,))

        ### Your code starts here:
        pass
        ### Your code ends here
        return loss


In [ ]:
# Test your implementation:
test_pred, test_gt = torch.zeros((1,1,5,5)), torch.zeros((1,1,5,5))
test_pred[0,0,0,:3] = 0.8
test_gt[0,0,0,2:] = 1

loss = SorensenDiceLoss()

if np.allclose(loss(test_pred, test_gt).item(), 0.67479676):
    print("Your implementation is correct!")
else:
    print("There is some problem in your implementation")

### b) Training a UNet model
Some information about the code that is provided:

- In `cvf20/transforms.py` and `cvf20/metrics.py` you can find the data augmentation functions and the metrics that you implemented in the last assignment.
- In `cvf20/utils.py` you can find a function to normalize the data. We will use the first 14 images for training and the last 4 for validation.
- The implementation of the `UNet` model is in `cvf20/models/UNet.py`. In the code block below you find an example of basic UNet model with depth 5 and the correct number of input/output channels needed for our foreground/background task.

##### Task 1.) Creating the data loaders
First, let's create the data loaders as we did in the last assignment

In [ ]:
from cvf20.utils import normalize_dataset
from cvf20.datasets import YeastCellDataset
import cvf20.transforms as T
from torch.utils.data.dataloader import DataLoader

# Compose and normalize the data in a .hdf5 file:
normalize_dataset()

# Add the transformations we used last time:
all_transforms = T.Compose(
    T.RandomFlip(),
    T.RandomRotation(),
    T.ToTorchTensor()
)

# For training, we choose a stride = (64,64). In this way during an epoch the same portion of an image are
# seen multiple times, but we make sure that some parts are not always feeded at the border of the 
# training (512,512) window. 
# In your experiments you can tweak this parameter (smaller value equal to more iterations in one epoch):
train_dataset = YeastCellDataset('./yeast_cells_dataset/dataset.hdf5',
                          (512,512),
                          (64,64),
                          mode="train",
                          transforms=all_transforms
                        )

# For validation, we make sure to visit the data only once (so we set stride=(512,512)):
val_dataset = YeastCellDataset('./yeast_cells_dataset/dataset.hdf5',
                          (512,512),
                          (512,512), 
                          mode="val",
                          transforms=all_transforms
                        )

# Create the data loaders:
train_loader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        drop_last=True,
        num_workers=2
)
val_loader = DataLoader(
        val_dataset,
        batch_size=4,
        shuffle=True,
        drop_last=True,
        num_workers=2
)


##### Task 2.) Using a GPU
Then, let's check if CUDA is available, i.e. if we can train on a GPU:

In [ ]:
# We will be using float throughout this tutorial
dtype = torch.float32 

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print("Warning, GPU not available! Please make sure to use one otherwise the training will be VERY slow")

print('Using device:', device)

##### Task 3.) The Trainer Class
In the next code block you will find the `Trainer` class that you will use to train your model. It includes code to perform the training iterations, compute the loss, update the parameters in the neural network, evaluate the metrics and log data during training.

Considering the goal of this exercise, it is not strictly needed to understand every line of code in it. But if you are eager to learn more, the code below will give you a good idea of how the PyTorch mechanics work.

In [ ]:
from cvf20.metrics import compute_accuracy, compute_IoU
from tensorboardX import SummaryWriter

class Trainer(object):
    def __init__(self, experiment_name, model,
                 optimizer, loss_function,
                 loader_train, loader_val,
                 dtype, device,
                 print_every=200, validate_every=100):
        """
        :param experiment_name: Name of the experiment. A folder with the name 'experiments/experiment_name` will be
                created with all the data associated to this run.

        :param model: PyTorch model of the neural network

        :param optimizer: PyTorch optimizer

        :param print_every: How often should we print the loss during training (and send some training plots to
                tensorboard)

        :param validate_every: How often (after how many training iterations) should we evaluate the results on the
                validation set (and send some validation plots to tensorboard)
        """
        # Create attributes:
        self.model = model
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.loader_train = loader_train
        self.loader_val = loader_val
        self.validate_every = validate_every
        self.print_every = print_every
        self.device = device
        self.dtype = dtype

        # Create experiment directory:
        exp_path = os.path.join('experiments', experiment_name)
        os.makedirs(exp_path, exist_ok=True)

        # Create Tensorboard logger:
        self.writer = SummaryWriter(exp_path)

    def train_model(self, epochs=1):
        """
        - epochs: (Optional) An integer giving the number of epochs to train for
        """
        model = self.model.to(device=self.device)  # move the model parameters to CPU/GPU
        nb_iter_per_epoch = 0
        for e in range(epochs):
            for t, (input, target) in enumerate(self.loader_train):
                model.train()  # put model to training mode
                input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
                target = target.to(device=self.device, dtype=self.dtype)

                prediction = model(input)
                loss = self.loss_function(prediction, target)

                # Zero out all of the gradients for the variables which the optimizer
                # will update.
                self.optimizer.zero_grad()

                # This is the backwards pass: compute the gradient of the loss with
                # respect to each  parameter of the model.
                loss.backward()

                # Actually update the parameters of the model using the gradients
                # computed by the backwards pass.
                self.optimizer.step()

                # Make sure that we apply a final activation if it was not done already:
                if self.model.final_activation is None:
                    print(prediction.max().item(), prediction.min().item())
                    prediction = torch.sigmoid(prediction)

                # Compute metrics:
                accuracy = compute_accuracy(prediction, target)
                IoU = compute_IoU(prediction, target)

                # Log some data to tensorboard:
                self.writer.add_scalar('loss_train', loss.item(), t + e * nb_iter_per_epoch)
                self.writer.add_scalar('IoU_train', IoU.item(), t + e * nb_iter_per_epoch)
                self.writer.add_scalar('accuracy_train', accuracy.item(), t + e * nb_iter_per_epoch)

                if t % self.print_every == 0:
                    self.make_plots(input, prediction, target, t + e * nb_iter_per_epoch, "predictions_train")
                    string1 = f'Epoch {e + 1}, iter {t}'
                    string2 = f'Loss: {loss.item()}'
                    print('{:<25s} ---> \t{:<30s}'.format(string1, string2))

                if t % self.validate_every == 0:
                    self.evaluate_metrics_on_val_set(t + e * nb_iter_per_epoch)

                # Increase counter:
                if e == 0:
                    nb_iter_per_epoch += 1

    def evaluate_metrics_on_val_set(self, global_step=None):
        # Set model to evaluation mode:
        # this is very important because some types of layers (for example BatchNorm) behave differently
        # during training and during evaluation.
        self.model.eval()

        # From now on, we make sure that torch does store data for computing gradients, since we won't
        # update the parameters of the model during validation. This makes the computations faster and
        # uses much less GPU memory.
        with torch.no_grad():
            # During validation, we accumulate these values across the whole dataset and then average at the end:
            accuracy, IoU, loss = 0., 0., 0.
            nb_iter = 0
            for input, target in self.loader_val:
                input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
                target = target.to(device=self.device, dtype=self.dtype)
                prediction = self.model(input)
                loss = loss + self.loss_function(prediction, target)

                # Make sure that we apply a final activation if it was not done already:
                if self.model.final_activation is None:
                    prediction = torch.sigmoid(prediction)

                accuracy = accuracy + compute_accuracy(prediction, target)
                IoU = IoU + compute_IoU(prediction, target)
                if nb_iter == 0:
                    self.make_plots(input, prediction, target, global_step, name_figure="predictions_val")
                nb_iter += 1
            
            loss = loss / nb_iter
            IoU = IoU / nb_iter
            accuracy = accuracy / nb_iter
            if global_step is not None:
                # Log scores averaged over all the valid set (send them to tensorboard):
                self.writer.add_scalar('loss_validation', loss.item(), global_step)
                self.writer.add_scalar('IoU_validation', IoU.item(), global_step)
                self.writer.add_scalar('accuracy_validation', accuracy.item(), global_step)
            else:
                # Print the results and return them:
                print("Validation loss function: ", loss.item())
                print("Validation IoU: ", IoU.item())
                print("Validation accuracy: ", accuracy.item())
                return loss.item(), IoU.item(), accuracy.item() 

    def make_plots(self, input, predictions, targets,
                   step, name_figure="image_log"):
        # First, we need to move the data back to CPU:
        input = input.cpu().detach().numpy()
        predictions = predictions.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()

        # Then we create some plots
        f, axes = plt.subplots(ncols=4, nrows=predictions.shape[0], figsize=(8, 8))
        for ax in axes.flatten():
            ax.axis('off')  # Delete axes
        axes[0, 0].set_title("Input image")
        axes[0, 1].set_title("Yeast-cell\nprediction")
        axes[0, 2].set_title("Ground truth")
        axes[0, 3].set_title("Pixels not\ncorrectly classified")
        for btc in range(predictions.shape[0]):
            axes[btc, 0].imshow(input[btc, 0], cmap='gray')
            axes[btc, 1].imshow(predictions[btc, 0], cmap='gray', vmin=0, vmax=1)
            axes[btc, 2].imshow(targets[btc, 0], cmap='gray', vmin=0, vmax=1)
            axes[btc, 3].imshow((predictions[btc, 0] > 0.5) == targets[btc, 0], cmap='seismic_r')
        plt.tight_layout()  # Reduce padding between subplots

        self.writer.add_figure(name_figure, f, step)  # Send the plot to tensorboard


##### Task 4.) Your first experiment!
In the next block we create a UNet model, build the Adam optimizer that will take care of updating the parameters and then start the training. You can choose an `experiment_name` to be passed to the `Trainer` class, so that you will find all the data related to it in the folder `experiments/experiment_name`. 

**Remark about the number of epochs:** Observing the loss plots during the first epoch is already enough to see if there are some bugs in your implementation. Running scripts on Google Colab seems to take longer than on a local GPU, but after few epochs (max 5) you should already be able to draw your conclusions. On a local GPU, 10 epochs will take approximately one hour. In the bonus exercise, you can decide to let your best model train longer to see how good it can get.

In [ ]:
### NAME EXPERIMENT: first experiment with Soresen-Dice Loss ###

from cvf20.models.unet import UNet

# Build a basic UNet model:
starting_model = UNet(
     depth=5,
     in_channels=1,
     out_channels=1,
     fmaps=(16, 32, 64, 128, 512, 1024),
     dim=2,
     scale_factor=2,
     activation=nn.ReLU,
     final_activation=nn.Sigmoid
)

# Build the optimizer:
params = starting_model.parameters()
learning_rate = 1e-4
optimizer = torch.optim.AdamW(params, lr=learning_rate)

# Build the trainer with the Soresen-Dice loss you implemented:
trainer = Trainer('first_exp_diceLoss', starting_model, optimizer, SorensenDiceLoss(),
        train_loader, val_loader, dtype, device)

# Start training:
trainer.train_model(epochs=1)
final_scores = trainer.evaluate_metrics_on_val_set()

##### Task 5.) Have a look at what it was logged in Tensorboard!
By running the following code block, as an output you will get a link that you can use to see tensorboard in a separate webpage (if you are using Google Colab). If instead you are using your local conda environment, then you can find tensorboard at [http://localhost:6006](http://localhost:6006).

In Tensorboard, you will see two icons in the upper orange bar: `Scalars` (showing scores and value of the loss) and `Images`.

If you are not happy with one experiment and you want to delete the data from Tensorboard, just delete the folder `experiments/experiment_name` containing its data (for example by running `!rm -r ./experiments/experiment_name`)


In [ ]:
### RUN THIS CODE TO START TENSORBOARD ###
LOG_DIR = './experiments'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
### RUN THIS CODE IF YOU USE GOOGLE COLAB AND YOU WANT TO SEE TENSORBOARD IN ANOTHER WEBPAGE (output link) ###
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
### RUN THIS CODE IF YOU USE GOOGLE COLAB AND YOU WANT TO SEE TENSORBOARD HERE IN THE NOTEBOOK ###
# Remark: This method is less preferred because this embedded interface seems to be less responsive 
# and it does not allow to download images of the plots
%load_ext tensorboard
%tensorboard --logdir experiments

##### Task 6.) Binary Cross Entropy Loss
Now run another experiment using this different type of loss. With this loss function, sometimes the training works  better if you remove the final `Sigmoid` activation from the model and use the loss `torch.nn.BCEWithLogitsLoss`, which combines a Sigmoid layer and the `torch.nn.BCELoss` in one single class to avoid outputs with infinite values.


In [ ]:
### NAME EXPERIMENT: second experiment with BCE Loss ###

from cvf20.models.unet import UNet

### Your code starts here (see first experiments above) ###
pass
### Your code ends here ###

*Your answer goes here*

Do not forget to report the achived scores and comment them. You can also download some plots in tensorboard or take screenshots of the plots shown there to support your comments. You can either load these figures here in the notebook, point us to their path in your `.zip` submission file, or comment them in a seperate `LaTex` file.

##### Task 7.) Add normalization layers
Repeat the previous two experiments by adding normalization layers to the UNet model.

In [ ]:
### NAME EXPERIMENT: ... ###

from cvf20.models.unet import UNet

### Your code starts here (see first experiments above) ###
pass
### Your code ends here ###

### c) Get creative!
Now run your additional experiments and report your results using the same scheme described above. Don't forget to explain what you implemented and point us to your code (if it is not included in this notebook).

You can find documentation for all the neural network layers implemented in PyTorch at [this link](https://pytorch.org/docs/stable/nn.html) (layer categories are on the right).

##### Exp. 1: Description
*Insert description here*

In [ ]:
### NAME EXPERIMENT: your experiment ###

from cvf20.models.unet import UNet

your_model = None

# Build the optimizer:
pass

# Build the trainer:
pass

# Start training:
pass
final_scores = trainer.evaluate_metrics_on_val_set()

##### Exp.1: Comments and Results
*Insert description here*

##### Exp. 2: Description
*Insert description here*

In [ ]:
### NAME EXPERIMENT: your experiment ###

from cvf20.models.unet import UNet

your_model = None

# Build the optimizer:
pass

# Build the trainer:
pass

# Start training:
pass
final_scores = trainer.evaluate_metrics_on_val_set()

##### Exp.2: Comments and Results
*Insert description here*